# pl

> Phase Linking

<!-- Phase linking for PTA requires `cupy.optimize` to be implented which is tracked at
https://github.com/rapidsai/raft/pull/923
and https://github.com/cupy/cupy/issues/6112 -->

In [ ]:
#| default_exp pl

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
# import for test
import numpy as np

In [ ]:
#| export
import cupy as cp

In [ ]:
#| export
def emi(coh:cp.ndarray, #coherence ,dtype cupy.complex
       )-> cp.ndarray: # estimated phase history `ph`, dtype complex
    coh_mag = abs(coh)
    coh_mag_inv = cp.linalg.inv(coh_mag)
    min_eig = cp.linalg.eigh(coh_mag_inv*coh)[1][...,0]
    return min_eig/abs(min_eig)

`emi` is a phase estimator base on EMI [@ansariEfficientPhaseEstimation2018] phase linking method.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()